<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/TPOT%20-%20AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPOT Tutorial
* Machine Learning automático.

> Os dados estão relacionados com campanhas de marketing direto (realizadas por telefone) de uma instituição bancária portuguesa. Freqüentemente, foi necessário mais de um contato para o mesmo cliente, para se verificar se o produto foi adquirido ('yes') ou não ('no').

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

In [0]:
!pip install tpot

     |████████████████████████████████| 901kB 4.8MB/s 
     |████████████████████████████████| 153kB 48.5MB/s 
  Created wheel for tpot: filename=TPOT-0.10.2-cp27-none-any.whl size=75534 sha256=f369349eb379b241c20364bfe63ae0d62026206d94439bf2d504908ea04bbf75
  Stored in directory: /root/.cache/pip/wheels/ca/9c/3e/5e1a3dc59d666aeecd4104992cab01bd8d2454e4d5b40057a4
  Created wheel for stopit: filename=stopit-1.1.2-cp27-none-any.whl size=11956 sha256=c442420a996f4a72fe573dfb0e6366afef45e674b0a394cad6db14a78f37e3ad
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built tpot stopit


In [0]:
# Importar as bibliotecas
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

In [113]:
# Carregar os dados
url= 'https://raw.githubusercontent.com/MathMachado/Python_RFB/DS_Python/Dataframes/bank-full.csv?token=AGDJQ622LYCH5FJDMIEHDM25QSDN4'
df_Mkt=pd.read_csv(url, sep= ';')
df_Mkt.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# Exploração dos Dados

In [114]:
df_Mkt.groupby(['loan']).y.value_counts().unstack()

y,no,yes
loan,,
no,33162,4805
yes,6760,484


In [115]:
df_Mkt.groupby(['loan','marital']).y.value_counts().unstack()

y                 no   yes
loan marital              
no   divorced   3731   555
     married   20069  2485
     single     9362  1765
yes  divorced    854    67
     married    4390   270
     single     1516   147

# Manipulação dos Dados

* TPOT exige que a variável-resposta seja renomeada para 'class';

In [0]:
df_Mkt.rename(columns={'y': 'class'}, inplace=True)

> TPOT exige que todos os dados estejam em formato numérico.
    >> Como podemos ver abaixo, o dataframe contem 11 variáveis categóricas (valores não-numéricos): job, marital, education, default, housing, loan, contact, month, day_of_week, poutcome, class.

In [117]:
df_Mkt.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
class        object
dtype: object

In [118]:
# Lista de variáveis categóricas do banco de dados
l_Cat_Vars= list(df_Mkt.select_dtypes(include=['object']).columns)
l_Cat_Vars

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome',
 'class']

Em seguida, verificamos o número de níveis que cada uma das cinco variáveis categóricas possui.

* Codificamos as categorias manualmente para numéricos.
* NaN's foram substituidos pela mediana.

In [0]:
df_Mkt['marital']= df_Mkt['marital'].map({'married':0,'single':1,'divorced':2,'unknown':3})
df_Mkt['default']= df_Mkt['default'].map({'no':0,'yes':1,'unknown':2})
df_Mkt['housing']= df_Mkt['housing'].map({'no':0,'yes':1,'unknown':2})
df_Mkt['loan']= df_Mkt['loan'].map({'no':0,'yes':1,'unknown':2})
df_Mkt['contact']= df_Mkt['contact'].map({'telephone':0,'cellular':1})
df_Mkt['poutcome']= df_Mkt['poutcome'].map({'nonexistent':0,'failure':1,'success':2})
df_Mkt['class']= df_Mkt['class'].map({'no':0,'yes':1})

In [120]:
df_Mkt = df_Mkt.fillna(df_Mkt.median())
pd.isnull(df_Mkt).any()

age          False
job          False
marital      False
education    False
default      False
balance      False
housing      False
loan         False
contact      False
day          False
month        False
duration     False
campaign     False
pdays        False
previous     False
poutcome     False
class        False
dtype: bool

* Para outras variáveis categóricas, usei pd.get_dummies...

In [121]:
# Lista de variáveis categóricas do banco de dados
l_Cat_Vars= list(df_Mkt.select_dtypes(include=['object']).columns)
l_Cat_Vars

['job', 'education', 'month']

In [122]:
# Constroi variáveis dummy
df_Mkt = pd.get_dummies(df_Mkt)
df_Mkt.head()

,age,marital,default,balance,housing,loan,contact,day,duration,campaign,pdays,previous,poutcome,class,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,education_primary,education_secondary,education_tertiary,education_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,58,0,0,2143,1,0,1.0,5,261,1,-1,0,1.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,44,1,0,29,1,0,1.0,5,151,1,-1,0,1.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,33,0,0,2,1,1,1.0,5,76,1,-1,0,1.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,47,0,0,1506,1,0,1.0,5,92,1,-1,0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,33,1,0,1,0,0,1.0,5,198,1,-1,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [123]:
np.isnan(df_Mkt).any()

age                    False
marital                False
default                False
balance                False
housing                False
loan                   False
contact                False
day                    False
duration               False
campaign               False
pdays                  False
previous               False
poutcome               False
class                  False
job_admin.             False
job_blue-collar        False
job_entrepreneur       False
job_housemaid          False
job_management         False
job_retired            False
job_self-employed      False
job_services           False
job_student            False
job_technician         False
job_unemployed         False
job_unknown            False
education_primary      False
education_secondary    False
education_tertiary     False
education_unknown      False
month_apr              False
month_aug              False
month_dec              False
month_feb              False
month_jan     

Tendo em mente que os dados estarão no formato numpy, podemos verificar o número de recursos no conjunto de dados final da seguinte forma.

In [0]:
y= df_Mkt['class']
X= df_Mkt.drop(['class'], axis= 1)

# Convertendo para numpy
X= X.to_numpy()
y= y.to_numpy()

In [132]:
np.shape(X)

(45211, 41)

In [126]:
df_Mkt.shape

(45211, 42)

# Análise de Dados com TPOT

* Dividir os dados em conjuntos de treinamento e validação.

In [144]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)
np.shape(X_train), np.shape(y_train)

((36168, 41), (36168,))

Agora passamos a chamar as funções fit(), score() e export() em nosso conjunto de dados de treinamento. Um parâmetro TPOT importante a ser definido é o número de gerações (via kwarg de gerações). Como nosso objetivo é apenas ilustrar o uso do TPOT, assumimos a configuração padrão de 100 gerações, limitando o tempo total de execução através do max_time_mins. Além disso, habilitamos o controle do tempo máximo permitido para a otimização de um único pipeline, via max_eval_time_mins.

Em um laptop padrão com 4 GB de RAM, cada geração leva aproximadamente 5 minutos para ser executada. Portanto, para o valor padrão de 100, sem a duração explícita vinculada, o tempo total de execução pode ser de aproximadamente 8 horas.

In [145]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2, max_eval_time_mins=0.04, population_size=15)
tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.90021551991
Generation 2 - Current best internal CV score: 0.900381479677

2.03246738333 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=9, min_samples_leaf=14, min_samples_split=9)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
        disable_update_check=False, early_stop=None, generations=1000000,
        max_eval_time_mins=0.04, max_time_mins=2, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=15,
        random_state=None, scoring=None, subsample=1.0, template=None,
        use_dask=False, verbosity=2, warm_start=False)

Acima, quatro gerações foram computadas, cada uma fornecendo a eficiência de treinamento do modelo de ajuste no conjunto de treinamento. Como é evidente, o melhor pipeline é aquele que tem a pontuação CV de 91,373%. O modelo que produz esse resultado é o algoritmo de árvore de decisão. Em seguida, o erro de teste é calculado para fins de validação.

# Exercícios
* Em grupo de até 4 alunos, estude e aplique as técnicas de Machine Learning que aprendemos até aqui: TPOT, Ensemble e etc para entregar o melhor modelo possível.

## Exercício 1 - Big Mart Sales
* Big_Mart_Sales_III
    * [Dados](https://raw.githubusercontent.com/MathMachado/Python_RFB/master/Dataframes/Big_Mart_Sales_III_train.txt?token=AGDJQ6ZOMVTZUMU4K3F7DW25QSRJ6);
    * Variável-target: Item_Outlet_Sales.

## Exercício 2 - Titanic
* Dados: Titanic_Original.csv que está na pasta Dataframe.

## Exercício 3 - Predizer se um indívidio ganha mais que $50k
* https://archive.ics.uci.edu/ml/datasets/Adult

* Base de dados: adult.csv que está no repositório dataframes.